In [11]:
import subprocess, msprime, pyslim, tskit, os, matplotlib
from scipy.integrate import quad
from matplotlib import pyplot as plt
import numpy as np
import itertools

In [12]:
os.chdir('/project2/jnovembre/ccliu/backgroundSelection/')

In [13]:
def allele_counts(ts, sample_sets=None):
    if sample_sets is None:
        sample_sets = [ts.samples()]
    def f(x):
        return x
    return ts.sample_count_stat(sample_sets, f, len(sample_sets),
               span_normalise=False, windows='sites',
               polarised=True, mode='site', strict=False)


def get_afss(ts_file, sample_size = None):
    if sample_size:
        ts = pyslim.load(ts_file).simplify()
        keep_indivs = np.random.choice(ts.individuals_alive_at(0), 
                                       sample_size, replace=False)
        keep_nodes = []
        for i in keep_indivs:
            keep_nodes.extend(ts.individual(i).nodes)
        ts = ts.simplify(keep_nodes)
        #print(ts.pairwise_diversity())

    mut_type = np.zeros(ts.num_sites)
    for j, s in enumerate(ts.sites()):
        mt = []
        for m in s.mutations:
            for md in m.metadata:
                mt.append(md.mutation_type)
        if len(set(mt)) > 1:
            mut_type[j] = 4
        else:
            mut_type[j] = mt[0]

    freqs = allele_counts(ts)
    l = len(ts.samples())
    freqs = freqs.flatten().astype(int)
    mut_afs = np.zeros((l + 1, 4), dtype='int64')
    for k in range(4):
        mut_afs[:, k] = np.bincount(freqs[mut_type == k+1], 
                                    minlength=l + 1)
    
    return mut_afs[1:(2 * sample_size),:]


def theta_pi(sfs):
    n = len(sfs)
    num = (np.array(range(1, n + 1)) * np.array(range(n, 0, -1)) * sfs).sum()
    denum = n * (n + 1) / 2
    pi = num / denum
    return pi

def compute_diversity(ts_file):
    ts = pyslim.load(ts_file).simplify()
    return ts.pairwise_diversity()

In [14]:
Ne = 5000
mu = 1e-7
l = 5e5
Ud = mu * l
Un = mu * 5 * l

theta = 4 * Ne * Un 
s = -0.1
B = np.exp(Ud / s * 2)
print(f'Ne x s =  {- Ne * B * s}')
print(f'B =  {B}')

Ne x s =  183.9397205857212
B =  0.3678794411714424


## Estimate using SFS (pairwise-diversity)

In [15]:
theta

5000.0

In [41]:
pis = []
for i in range(1, 101):
    ts_file = f'data/slim/bgs_Bscore/bgs_BscoreN5000_rep{i}.trees'
    afss = get_afss(ts_file, 500)
    pis.append(theta_pi(afss[:,1]))
B_theta = np.array(pis).mean() / theta
std_B_theta = np.array(pis).std() / theta

In [42]:
print(B_theta)
print(std_B_theta)

0.3505923188388388
0.16555080969215055


## Estimate using pairwise-coalescent time

In [26]:
def avg_coal_time(tree):  
    '''
    Another idea for computing pairwise-coalescent time 
    from Xinyi Li; not used here
    '''
    coalescence_time = 0
    sample_size = tree.get_sample_size()
    for leaves in np.arange(sample_size+1):
        u = leaves
        while u != tskit.NULL:
            visited = np.array([])
            u = tree.parent(u)
            if u !=-1 : 
                node_children = np.array(tree.children(u))
                node_interest = np.setdiff1d(node_children, visited)
                num_leaves = tree.get_num_leaves(node_interest[0])
                node_branch_len = 2 * tree.time(u) * num_leaves
                coalescence_time = coalescence_time + node_branch_len
    return(coalescence_time/(sample_size * (sample_size-1)))

In [43]:
pairwise_coaltimes = []
for i in range(1, 101):
    pairwise_coaltime = []
    ts_file = f'data/slim/bgs_Bscore/bgs_BscoreN5000_rep{i}.trees'
    ts_full = pyslim.load(ts_file).simplify()
    for j in range(1, 1001):
        keep_indivs = np.random.choice(ts_full.individuals_alive_at(0),
                                       1,
                                       replace=False)
        keep_nodes = []
        for i in keep_indivs:
            keep_nodes.extend(ts_full.individual(i).nodes)
        ts = ts_full.simplify(keep_nodes)
        # pairwise coalescent time
        tree = ts.first()
        try:
            pairwise_coaltime.append(tree.time(tree.root))
        except:
            pass
    pairwise_coaltimes.append(np.array(pairwise_coaltime).mean() / 2)
    
pairwise_coaltimes = np.array(pairwise_coaltimes)
print(f'mean T2: {pairwise_coaltimes.mean()}')
B_coal = pairwise_coaltimes.mean() / 5000
std_B_coal = pairwise_coaltimes.std() / 5000

mean T2: 1807.6380399999998


In [44]:
print(B_coal)
print(std_B_coal)

0.36152760799999994
0.17346037335841732
